![CFD.ML v.20](<CFD.ML v2.0.png>)

CFD.ML is delivered via DNV WindFarmer's web API. It is a machine learning model that can predict turbine interaction loss factors, both wake and blockage correction losses together. Find out more in the [WindFarmer documentation](https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/CalcRef/TurbineInteractions/CFDML/CFDML.html).

With this notebook you can:
1. import the wind farm data model as a json file, exported from WindFarmer, or converted from Openwind json or other formats.
1. define calculation settings
1. run the calculation and diagnose errors
1. interogate and visualise the results, comparing them to those from traditional engineering models. 

In [ ]:
import os
import requests
import json
import time
import random
import asyncio
import math
import pandas as pd
from copy import deepcopy
from matplotlib import pyplot as plt
import numpy as np

from script_lib import cfdml_v2
from script_lib import api_calls
from script_lib import aep_results

# Import the API inputs as a json file
The easiest and most reliable way to web API inputs is via the WindFarmer desktop application. 
Export the annual energy production calculation API input settings from within your workbook using an in-app script. Edit the file path and call:

```Toolbox.ExportWindFarmerEnergyJson( @"C:\folder\my_aep_calculation_inputs.json" );```

If you wish to construct the inputs yourself, see the datamodel described in our [API documentation and OpenAPI specification](https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/WebAPI/Introduction/Introduction.html). 

Download the atmospheric conditions presets from [here](https://mysoftware.dnv.com/download/renewables/windfarmer/Downloads/cfdml_preset_atmospheric_conditions.json) [login with veracity account] and save it in this folder, or specify the `atmospheric_conditions_presets_file_path` to the downloaded JSON file below.

We can also asist with a script to convert the workbook JSON file exported from OpenWind to the WindFarmer format.

In [ ]:
input_folder = './'
projectId = 'TheBowl'

# Export AEP input settings from your Windfarmer workbook using an in-app script and calling Toolbox.ExportWindFarmerEnergyJson( input_data_file_path ):
input_json_file_path = os.path.join(input_folder, 'TheBowl_with_neighbours_api_inputs_from_wfa_desktop.json')

# The json file containing pre-set atmospheric conditions, available to WindFarmer customers
atmospheric_conditions_presets_file_path = os.path.join(input_folder, "default_atmospheric_conditions.json.")

inputs_file_name = os.path.join(input_folder, f'cfdmlv2api_inputs_{projectId}.json')
results_file_name = os.path.join(f'cfdmlv2api_results_{projectId}.json')
results_file_name_no_neighbours = os.path.join(f'cfdmlv2api_no_neighbours_results_{projectId}.json')
per_turbine_results_file_name = os.path.join(f'cfdmlv2api_per_turbine_results_{projectId}.csv')

# project Info:
projectInfo = { "projectId": "TheBowl",
                "siteLatitude": 36.5,
                "siteLongitude": -75.1} 

### load the workbook JSON inputs

In [ ]:
with open(input_json_file_path) as f:
    json_string = f.read()
    json_input = json.loads(json_string)
# Assign a color to each wind farm
farm_names = [farm["windFarmName"] if "windFarmName" in farm else f"Farm_{i}" for i, farm in enumerate(json_input["windFarms"])]
color_map = plt.get_cmap('tab20')
farm_colors = {name: color_map(i % 20) for i, name in enumerate(farm_names)}

turbines = []
for farm, farm_name in zip(json_input["windFarms"], farm_names):
    for turbine in farm["turbines"]:
        new_turbine = deepcopy(turbine)
        new_turbine["farm_name"] = farm_name
        turbines.append(new_turbine)
number_of_turbines = len(turbines)
print(f'The project contains {number_of_turbines} turbines accross {len(json_input["windFarms"])} wind farms')

fig1, ax = plt.subplots(figsize=(10,7))
for wt in turbines:
    color = farm_colors[wt["farm_name"]]
    ax.scatter(wt['location']['easting_m'], wt['location']['northing_m'], c=[color], label=wt["farm_name"])
    ax.set_xlabel("Easting [m]")
    ax.set_ylabel("Northing [m]")
    ax.set_aspect('equal', adjustable='box')
    ax.annotate(wt['name'], [wt['location']['easting_m'], wt['location']['northing_m']])
ax.set_title(f'{projectId}')

# Add legend with unique farm names
handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys(), title="Wind Farm")

# Choose model inputs 

**Note**: the part responsible for model settings in the input jsons contained in the ```input_json_file_path```  will get overwritten by the settings specified

In [ ]:
# specify the desired model combination
WAKE_MODEL_CHOICE = "CFDML" # EddyViscosity/ModifiedPark/TurbOPark/CFDML    # Note CFDML wakes calculates the full TILF (wakes + blockage)
BLOCKAGE_MODEL_CHOICE = "CFDML" # BEET/CFDML                                # Note if you choose CFDML wakes, you can only choose CFD.ML blockage. 
CALCULATE_EFFICIENCIES = True #True/False                                   # Calculating efficiencies gives you a full breakdown of wakes, blockage, neighbour wakes etc. but takes longer to compute
CFDML_VERSION = "2.6.0"
BLOCKAGE_APPLICATION_METHOD = "OnWindSpeed" # OnWindSpeed / OnEnergy
NUMBER_OF_DIRECTION_STEPS = 180

# For WAKE_MODEL_CHOICE = CFD.ML  we currently reccomend not applying any large wind farm correction in by-wind-speed extrapolation of CFD.ML results.
EXTRAPOLATE_CFDML_USING_LARGE_WIND_FARM_CORRECTION = False # True/False

# However, should you wish to use this notebook with a WAKE_MODEL_CHOICE of EddyViscosity, 
# you should define your large wind farm correction settings 
# The below values are defaults for an offshore project
lwf_paramters = { 
                    "baseRoughnessZ01": 0.0004,
                    "increasedRoughnessZ02": 0.0192,
                    "geometricWidthDiameters": 1.0,
                    "recoveryStartDiameters": 120.0,
                    "fiftyPercentRecoveryDiameters": 40.0
                }

## Load and select atmospheric condition pre-sets for your site
You will need to define a stablity rose, defining the proportion of conditions in each class for each direction sector

In [ ]:
with open(atmospheric_conditions_presets_file_path, "r") as f:
    atmosperhic_condition_presets = json.load(f)
atmosperhic_condition_presets_df = pd.DataFrame(atmosperhic_condition_presets).T
atmosperhic_condition_presets_df

The training has been performed over a wide range of wind farms and atmospheric conditions as can be seen in the plots below.
| Wind farm parameters | Atmopsheric conditions |
|-|-|
|![alt text](trainingset_metadata.png) |![alt text](trainingset_metadata_atmos.png)|

### Define the proportion of each atmospheric condition class by wind direction
Edit the direction sectors and proportion of data that falls in each atmospheric condition class for each. 

We support custom direction sectors, so no need to specify 12 equal sectors. 

If you do not want to vary the distribution by direction, simply add one bin, 0 to 360 e.g:
```
atmosphericConditionProbabilityDistribution = [{
    "fromDirection_degrees": 0,
    "toDirection_degrees":   360, 
    "atmosphericConditionClassIds": [stable_ID, unstable_ID],
    "probabilityForClasses": [0.6, 0.4]
}
]
```

In [ ]:
# selected atmospheric condition pre-sets to use in simulation: 
stable_ID = "onshore_stable_idealized"
unstable_ID = "onshore_unstable_idealized"

atmosphericConditionProbabilityDistribution = [{
    "fromDirection_degrees": 345,
    "toDirection_degrees":   115, 
    "atmosphericConditionClassIds": [stable_ID, unstable_ID],
    "probabilityForClasses": [0.6, 0.4]
},
{
    "fromDirection_degrees": 115,  
    "toDirection_degrees": 285, 
    "atmosphericConditionClassIds": [stable_ID, unstable_ID],
    "probabilityForClasses": [0.45, 0.55]
},
{
    "fromDirection_degrees": 285,
    "toDirection_degrees"  : 345,
    "atmosphericConditionClassIds": [stable_ID, unstable_ID],
    "probabilityForClasses": [0.5, 0.5]
},
]

In [ ]:
from script_lib import cfdml_v2
cfdml_v2.plot_atmoshperic_conditions_rose(atmosphericConditionProbabilityDistribution)
pd.DataFrame(atmosphericConditionProbabilityDistribution)

In [ ]:
# Import utility functions
from script_lib.input_json_prep import (
    set_model_settings, 
    switch_off_fpm_export, 
    configure_fpm_export,
    get_avg_hub_and_tip_heights_for_subject_windfarms,
    construct_atmospheric_conditions
)

# Defining the model inputs
set_model_settings(
    json_input,
    wake_model_choice=WAKE_MODEL_CHOICE,
    blockage_model_choice=BLOCKAGE_MODEL_CHOICE,
    calculate_efficiencies=CALCULATE_EFFICIENCIES,
    number_of_direction_steps=NUMBER_OF_DIRECTION_STEPS,
    cfdml_version=CFDML_VERSION,
    blockage_application_method=BLOCKAGE_APPLICATION_METHOD
)

switch_off_fpm_export(json_input)
configure_fpm_export(json_input)  # activate wind speed fpms for the flow case visualisations

hub_height, tip_height = get_avg_hub_and_tip_heights_for_subject_windfarms(json_input)
json_input["atmosphericConditions"] = construct_atmospheric_conditions(
    atmosphericConditionProbabilityDistribution, 
    atmosperhic_condition_presets, 
    hub_height, 
    tip_height
)
json_input["projectInfo"] = projectInfo

In [ ]:
# write inputs to file
inputs_file_name = f'cfdmlv2apiinputs_{projectInfo["projectId"]}.json' 
with open(inputs_file_name, 'w') as f:
    json.dump(json_input, f, indent=4, allow_nan=False)

## API setup and calling
To access the API you need a authorization token. 
This should be kept secure - and not added to source control, so I'm getting it from an environment variable. See setup instructions for saving your access key as an environment variable documented [here.](https://myworkspace.dnv.com/download/public/renewables/windfarmer/manuals/latest/WebAPI/Introduction/gettingStarted.html)

In [ ]:
api_url = 'https://windfarmer.dnv.com/api/v2/'
auth_token = os.environ['WINDFARMER_ACCESS_KEY']

# The token should be passed as an Authorization header.
#  We also need to set the `Content-Type` to let the API know that we're sending JSON data.

wf_api = api_calls.WindFarmerAPI(auth_token, api_url)

### Call `AnnualEnergyProduction` endpoint to calculate results

In [ ]:
# reload inputs from file
with open(inputs_file_name, 'r') as f:
    json_input = json.load(f)

json_input_no_neighbours = cfdml_v2.generate_no_neighbours_inputs(json_input)

In [ ]:
# Make the API call, 
if cfdml_v2.check_if_neighbours(json_input):
    api_response_no_neighbours = await wf_api.call_aep_api( json_input_no_neighbours)
else:
    api_response_no_neighbours = None
api_response = await wf_api.call_aep_api( json_input)

In [ ]:
# # If want to try getting results after interupting the script, 
# # you can uncomment the below, enter your job ID printed above, then try and get the results

# job_id = "772b948e-f31e-49aa-9684-c3ab26d43e85"
# status, api_response = await wf_api.poll_for_status(job_id)

In [ ]:
with open(results_file_name, 'w') as f:
    f.write(json.dumps(api_response, indent=4))

if cfdml_v2.check_if_neighbours(json_input):
    with open(results_file_name_no_neighbours, 'w') as f:
        f.write(json.dumps(api_response_no_neighbours, indent=4))

time.sleep(1)

# Review results
### Report Gross + Full AEP + Wake + Blockage loss factors

Read results back in from file (in case you wish to run plotting and reporting again without recalculation )

In [ ]:
with open(results_file_name, 'r') as f:
    results = json.load(f)
if cfdml_v2.check_if_neighbours(json_input):
    with open(results_file_name_no_neighbours, 'r') as f:
        results_no_neighbours = json.load(f)
else:
    results_no_neighbours = None

In [ ]:
results_processor = aep_results.AEPResultsProcessor(json_input, results, results_no_neighbours)
results_summary_df = results_processor.get_results_summary_df()
results_summary_df.T    

Note:
* Total turbine interaction efficiency = blockage * internal_wake * external_wake
* The total losses can also include hysteresis and curtailment efficiencies, depending on the chosen calculation settings

## Pattern of production
The annual energy production pattern accross the farm

In [ ]:
# Pattern of production - Full yield
per_turbine_results = pd.DataFrame.from_dict(results['windFarmAepOutputs'][0]['turbineResults'])
per_turbine_results.set_index('turbineName', drop=True, inplace=True)
per_turbine_results['easting'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['easting_m'])
per_turbine_results['northing'] = per_turbine_results['turbineLocation'].map(lambda x: dict(x)['northing_m'])
fig, ax = plt.subplots(figsize=(10,4))
im = ax.scatter(per_turbine_results['easting'], per_turbine_results['northing'], c=per_turbine_results['fullAnnualYield_MWh_per_year'], cmap='viridis')
ax.set_title("Per turbine energy production (blockage & wakes affected) [MWh/yr]")
ax.set_xlabel("Easting [m]")
ax.set_ylabel("Northing [m]")
ax.set_aspect('equal', adjustable='box')
fig.colorbar(im)

## TILF pattern for selected wind directions

In [ ]:
# Select the wind direction and wind speed flow case you wish to review:
wind_direction = 220
wind_speed = 8.0

In [ ]:
def get_atmos_classes_for_direction (wind_direction, atmosphericConditionProbabilityDistribution):
    for sector in atmosphericConditionProbabilityDistribution:
        to_direction = sector["toDirection_degrees"]
        from_direction = sector["fromDirection_degrees"]
        if  from_direction > to_direction :
            to_direction = to_direction + 360.0
            test_wind_direction = wind_direction + 360.0
        if (from_direction <= test_wind_direction and to_direction > wind_direction):
            classes_for_direction = sector["atmosphericConditionClassIds"]
            print(f"atmospheric conditions {classes_for_direction} found for wind direction {wind_direction}")
            return classes_for_direction
    print ("No atmospheric conditions found for wind direction!")
    return None

atmos_classes_for_direction = get_atmos_classes_for_direction(wind_direction, atmosphericConditionProbabilityDistribution)

In [ ]:
# wind speed turbine interaction loss factor plotting accross array
wind_direction_index = int(wind_direction/ (360.0 / NUMBER_OF_DIRECTION_STEPS))
wind_speed_index = int(wind_speed)

waked_windspeed_cfdml = pd.DataFrame(results['windFarmAepOutputs'][0]['turbineFlowAndPerformanceMatricesWithMastBinning'])
waked_windspeed_cfdml.set_index('turbineName',drop=True, inplace=True)
for atmos_class in atmos_classes_for_direction:
    waked_windspeed_cfdml['atmosphericConditionAwareWakedWindSpeed_m_per_s{}'.format(atmos_class)] = waked_windspeed_cfdml['atmosphericConditionAwareWakedWindSpeed_m_per_s'].map(lambda x: x[atmos_class][wind_direction_index][wind_speed_index])

# Plot
waked_windspeed_cfdml['northing'] = per_turbine_results['northing']
waked_windspeed_cfdml['easting'] = per_turbine_results['easting']

vmin = 1.0
for atmos_class in atmos_classes_for_direction:
    waked_windspeed_cfdml['TILF {}'.format(atmos_class)] = waked_windspeed_cfdml['atmosphericConditionAwareWakedWindSpeed_m_per_s{}'.format(atmos_class)] / wind_speed
    bl_vmin = waked_windspeed_cfdml['TILF {}'.format(atmos_class)].min()
    if bl_vmin < vmin:
        vmin = bl_vmin
for atmos_class in atmos_classes_for_direction:
    fig3, ax3 = plt.subplots(figsize=(10,4))
    im2 = ax3.scatter(waked_windspeed_cfdml['easting'], waked_windspeed_cfdml['northing'], c=waked_windspeed_cfdml['TILF {}'.format(atmos_class)], cmap='viridis', vmin=vmin, vmax=1.0)
    fig3.colorbar(im2)
    ax3.set_title(f"TILF - CFD.ML\n Free stream: {wind_speed} m/s, {wind_direction} deg\nAtmospheric condition class: {atmos_class}")
    ax3.set_xlabel("Easting [m]")
    ax3.set_ylabel("Northing [m]")
    ax3.set_aspect('equal', adjustable='box')

In [ ]:
# Plot wind speed pattern for user specified turbines
northern_edge = ['T14', 'T10', 'T6', 'T3', 'T1', 'T2', 'T4', 'T9', 'T26', 'T33', 'T53', 'T74', 'T96']
#southern_edge = ['T95', 'T104', 'T99', 'T107', 'T102', 'T110', 'T97', 'T105', 'T100', 'T108', 'T103', 'T98', 'T106', 'T101', 'T109', 'T96']

fix, ax = plt.subplots(figsize=(7,4))
for atmos_class in atmos_classes_for_direction:
    (waked_windspeed_cfdml.loc[northern_edge]['atmosphericConditionAwareWakedWindSpeed_m_per_s{}'.format(atmos_class)]).plot(marker='o', ax=ax )
    handles, labels = ax.get_legend_handles_labels()
    ax.set_title(f'TILF CFD.ML at selected turbines\n Free stream: {wind_speed} m/s, {wind_direction} deg')
    ax.set_ylabel('Normalised Power')
    ax.set_xlabel('turbine label')
ax.legend(handles, atmos_classes_for_direction)
